# 02/2025
# Spisy z Postovej Banky
# Mame vela 'INE', ktore nie su az tak dolezite.
# Mame vela s viacerymi prilohami, ktore je OK poslat na manual.
# Mame dorucenky, ktore nevieme dalej triedit a posielame na manual.

# Chceme kategoriu INE, MULTIDOCUMENT (vsetko v kat2 s ozcnacenim 'viacero priloh'), DORUCENKA (vsetky dorucenky).
# Toto bude hlavny model. Dalsie triedenie je optional.


In [ ]:
import mltools
import mlflow
mlflow.set_tracking_uri("http://localhost:4000")

In [ ]:
import pandas as pd
import mltools
from copy import copy

# Get raw data
raw_df = pd.read_parquet('/Users/ondrejgutten/Work/PISI.nosync/data/PB/SpisyPB18-PB24_v3.parquet')

# preprocess texts
initial_preprocessor = mlflow.pyfunc.load_model('models:/InitialTextPreprocessor/1')
noplaceholders_preprocessor = mlflow.pyfunc.load_model('models:/InitialAndRemovePlaceholdersTextPreprocessor/1')

# Extract useful data (text + categories)
placeholders_df = copy(raw_df.iloc[:,[2,5]])
placeholders_df['Text'] = initial_preprocessor.predict(placeholders_df['Text'])

noplaceholders_df = copy(raw_df.iloc[:,[2,5]])
noplaceholders_df['Text'] = noplaceholders_preprocessor.predict(noplaceholders_df['Text'])

mydf = noplaceholders_df





In [ ]:
mydf = mltools.data.drop_classes_with_1_member(mydf,target_column = mydf.columns[1])
train_df, test_df = mltools.data.pandas_split_categorical_data(mydf)

X = mydf.loc[:,mydf.columns[0]]
y = mydf.loc[:,mydf.columns[1]]
train_X = train_df.loc[:,train_df.columns[0]]
train_y = train_df.loc[:,train_df.columns[1]]
test_X = test_df.loc[:,test_df.columns[0]]
test_y = test_df.loc[:,test_df.columns[1]]

In [ ]:
# Split data by 'Balik'

list_dfs_split_by_package = [df for df in mydf.groupby(raw_df['Balík'])]

# Mame baliky PB18a,PB18b,PB18c,PB21,PB24. V tomto poradi vyskusame pouzit prvych N ako trenovaci set a zvysok ako testovaci set
# Specifikujeme volbu stringom 'pa', 'pb','pc','p21','p24'

train_set_specification = 'p21'

In [ ]:

if train_set_specification == 'pa':
    train_set = [0]
    test_set = [1,2,3,4]
elif train_set_specification == 'pb':
    train_set = [0,1]
    test_set = [2,3,4]
elif train_set_specification == 'pc':
    train_set = [0,1,2]
    test_set = [3,4]
elif train_set_specification == 'p21':
    train_set = [0,1,2,3]
    test_set = [4]
elif train_set_specification == 'p24':
    train_set = [4]
    test_set = [0,1,2,3]
else:
    print('Invalid train_set_specification')
    exit()

train_X =pd.concat([list_dfs_split_by_package[i][1].iloc[:,0] for i in train_set]) 
train_y = pd.concat([list_dfs_split_by_package[i][1].iloc[:,1] for i in train_set])
test_X = pd.concat([list_dfs_split_by_package[i][1].iloc[:,0] for i in test_set])
test_y = pd.concat([list_dfs_split_by_package[i][1].iloc[:,1] for i in test_set])


# THINGS TO TRY:
- Single-label - multidoc ako samostatna kategoria
- Multilabel bez zlomov stran
- OCR quality ako prediktor zleho scanu? - Text uz obsahuje indikaciu zleho scanu
- nedelit dokumenty train_test splitom, ale podla balikov

In [ ]:
xgb = mltools.architecture.TF_IDF_XGBoost_Classifier('xgb',{})
xgb.fit(train_X, train_y)
model = xgb



In [ ]:
rf = mltools.architecture.TF_IDF_Vectorizer_RF('rf')
rf.fit(train_X, train_y)
model = rf

In [ ]:
# Quick evaluation
from sklearn.metrics import accuracy_score

y_pred = model.predict(test_X)
accuracy_score(test_y, y_pred)

# full train_test_split
# xgb = 0.7 
# RF = 0.999

# train_X = pa
# xgb = 0.81
# rf = 0.879

# train_X = pb
# rf = 0.856

# train_X = pc
# rf = 0.855

# train_X = p21
# rf = 0.898

# train_X = p24
# rf = 0.625

In [ ]:
mrf = mltools.architecture.TF_IDF_Multilabel_Classifier(rf,'multilabel')

In [ ]:
# Extract useful data (text + categories)
mydf = raw_df.iloc[:,[2,4]]

all_categories = mydf.iloc[:,1].unique()
single_doc_categories = [cat for cat in all_categories if '~' not in cat]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit([single_doc_categories])

X = mydf.iloc[:,0]
y = mlb.transform([x.split('~') for x in mydf.iloc[:,1]])

#mydf = mltools.data.drop_classes_with_1_member(mydf,target_column = mydf.columns[1])
train_df, test_df = mltools.data.pandas_split_categorical_data(mydf)

train_X = train_df.iloc[:,0]
train_y = mlb.transform([x.split('~') for x in train_df.iloc[:,1]])
test_X = test_df.iloc[:,0]
test_y = mlb.transform([x.split('~') for x in test_df.iloc[:,1]])

train_multidoc_bool = [sum(train_y[i]) > 1 for i in range(len(train_y))]
test_multidoc_bool = [sum(test_y[i]) > 1 for i in range(len(test_y))]

In [ ]:
rf.fit(train_X, train_y)

# multilabel je v test sade len 79 a ma accuracy 0.3.
# Presny hit je vzacny, ale v 74/79 pripadov su vsetky predikovane sucastou true labels,
# i.e. nizka accuracy je, lebo missneme niektore labels, ale nevymyslame si nove
# najcastejsie chyba dorucenka
# namatkovo 4679 z raw_df ma mat dorucenku, upozornenie pred vyzvou a hlavnu zmluvu - ale vyzera to len ako hlavna zmluva